<a href="https://colab.research.google.com/github/radsrei/Docs_Random/blob/Alura-RAG/Alura_RAG_Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# PROJETO PRÁTICO 2 — Agente Farmacêutico Explicador de Bulas (RAG)

**Curso:** LangChain — Criando Chatbots Inteligentes com RAG (Foundation)  
**Dificuldade:** Intermediário  

Este projeto representa a **evolução direta do Projeto 1**, incorporando:
- Múltiplos documentos
- Chunking mais estratégico
- Enriquecimento com metadados
- Maior controle semântico e rastreabilidade



## Dependências

```bash
pip install langchain==0.1.20 langchain-community==0.0.38 langchain-openai==0.1.7 chromadb pypdf
```


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip uninstall -y langchain langchain-core langchain-community langchain-openai

Found existing installation: langchain 0.1.20
Uninstalling langchain-0.1.20:
  Successfully uninstalled langchain-0.1.20
Found existing installation: langchain-core 0.1.53
Uninstalling langchain-core-0.1.53:
  Successfully uninstalled langchain-core-0.1.53
Found existing installation: langchain-community 0.0.38
Uninstalling langchain-community-0.0.38:
  Successfully uninstalled langchain-community-0.0.38
Found existing installation: langchain-openai 0.1.7
Uninstalling langchain-openai-0.1.7:
  Successfully uninstalled langchain-openai-0.1.7


In [5]:
# !pip install langchain==0.1.20 langchain-community==0.0.38 langchain-openai==0.1.7

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 169, in _make_candidate_from_dist
    base = self._installed_candidate_cache[dist.canonical_name]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
KeyError: 'sniffio'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-pack

In [4]:
# !pip install pypdf

In [1]:
# !pip install chromadb

In [2]:
# Importa o módulo para manipulação de variáveis de ambiente
import os

# Loader responsável por ler arquivos PDF
from langchain.document_loaders import PyPDFLoader

# Responsável por dividir textos grandes em chunks menores
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Classe que transforma texto em embeddings vetoriais
from langchain.embeddings import OpenAIEmbeddings

# Banco vetorial para armazenamento e busca semântica
from langchain.vectorstores import Chroma

# Modelo de linguagem conversacional
from langchain.chat_models import ChatOpenAI

# Cadeia pronta de Perguntas e Respostas com RAG
from langchain.chains import RetrievalQA


In [3]:

# Define a chave da API da OpenAI como variável de ambiente
# (Nunca versionar essa chave em repositórios públicos)
os.environ["OPENAI_API_KEY"] = ""



## 1️⃣ Definição do Problema

Bulas farmacêuticas possuem:
- Linguagem técnica
- Grande volume de texto
- Informações sensíveis

O objetivo do agente é **interpretar corretamente as bulas**,
respondendo perguntas **somente com base nos documentos**,
evitando qualquer tipo de alucinação.



## 2️⃣ Seleção e Organização das Bulas

Cada bula representa um medicamento diferente.
Utilizamos múltiplos PDFs como base de conhecimento.


In [4]:

# Lista com os caminhos dos arquivos PDF das bulas
caminhos_bulas = [
    "dipirona.pdf",
    "paracetamol.pdf"
]

# Lista que armazenará todos os documentos carregados
documentos = []

# Percorre cada bula
for caminho in caminhos_bulas:

    # Cria o loader do PDF
    loader = PyPDFLoader(caminho)

    # Carrega o conteúdo do PDF
    docs = loader.load()

    # Adiciona o nome do medicamento como metadado
    for doc in docs:
        doc.metadata["medicamento"] = caminho.split("/")[-1].replace(".pdf", "")

    # Adiciona os documentos à lista principal
    documentos.extend(docs)

# Exibe a quantidade total de páginas carregadas
len(documentos)


11

In [5]:
print(documentos)

[Document(page_content='  \n \n \n \n \n \n \n \n \ndipirona monoidratada \n \nBrainfarma Indústria Química e Farmacêutica S.A. \n \n \nComprimido  \n1g \n \n \n \n \n \n \n', metadata={'source': 'dipirona.pdf', 'page': 0, 'medicamento': 'dipirona'}), Document(page_content=' \ndipirona monoidratada – comprimido - Bula para o paciente  1 \n \nI - IDENTIFICAÇÃO DO MEDICAMENTO: \n \ndipirona monoidratada \nMedicamento genérico Lei nº 9.787, de 1999. \n \nAPRESENTAÇÕES \nComprimido. \nEmbalagens contendo 10 ou 100 comprimidos.  \n \nVIA DE ADMINISTRAÇÃO: ORAL \n \nUSO ADULTO E PEDIÁTRICO ACIMA DE 15 ANOS. \n \nCOMPOSIÇÃO \nCada comprimido contém: \ndipirona monoidratada ...................................................................................................................... 1000mg \nexcipientes q.s.p. ..................................................................................................................... 1 comprimido \n(macrogol, metabissulfito de sódio, dióxido d


## 3️⃣ Extração, Limpeza e Chunking

Dividimos o conteúdo das bulas em blocos menores,
preservando contexto por meio de sobreposição.


In [6]:

# Cria o objeto responsável pelo chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,      # Tamanho máximo de cada chunk
    chunk_overlap=150   # Sobreposição entre chunks
)

# Divide os documentos em chunks
chunks = text_splitter.split_documents(documentos)

# Quantidade total de chunks gerados
len(chunks)


61

In [7]:
chunks

[Document(page_content='dipirona monoidratada \n \nBrainfarma Indústria Química e Farmacêutica S.A. \n \n \nComprimido  \n1g', metadata={'source': 'dipirona.pdf', 'page': 0, 'medicamento': 'dipirona'}),
 Document(page_content='dipirona monoidratada – comprimido - Bula para o paciente  1 \n \nI - IDENTIFICAÇÃO DO MEDICAMENTO: \n \ndipirona monoidratada \nMedicamento genérico Lei nº 9.787, de 1999. \n \nAPRESENTAÇÕES \nComprimido. \nEmbalagens contendo 10 ou 100 comprimidos.  \n \nVIA DE ADMINISTRAÇÃO: ORAL \n \nUSO ADULTO E PEDIÁTRICO ACIMA DE 15 ANOS. \n \nCOMPOSIÇÃO \nCada comprimido contém: \ndipirona monoidratada ...................................................................................................................... 1000mg', metadata={'source': 'dipirona.pdf', 'page': 1, 'medicamento': 'dipirona'}),
 Document(page_content='dipirona monoidratada ...................................................................................................................... 1000mg 


## 4️⃣ Enriquecimento com Metadados

Os metadados permitem:
- Filtragem
- Explicabilidade
- Melhor relevância na recuperação


In [8]:

# Percorre cada chunk para classificar semanticamente seu conteúdo
for chunk in chunks:

    # Normaliza o texto para facilitar as verificações
    texto = chunk.page_content.lower()

    # Identificação do medicamento
    if "identificação do medicamento" in texto or "composição" in texto:
        chunk.metadata["categoria"] = "identificacao"

    # Indicações terapêuticas
    elif "indicação" in texto or "para que este medicamento é indicado" in texto:
        chunk.metadata["categoria"] = "indicacao"

    # Funcionamento do medicamento
    elif "como este medicamento funciona" in texto or "ação" in texto:
        chunk.metadata["categoria"] = "como_funciona"

    # Contraindicações
    elif "contraindicação" in texto or "quando não devo usar" in texto:
        chunk.metadata["categoria"] = "contraindicacao"

    # Advertências e precauções
    elif "advertência" in texto or "precaução" in texto or "o que devo saber antes de usar" in texto:
        chunk.metadata["categoria"] = "advertencias_precaucoes"

    # Interações medicamentosas
    elif "interação" in texto or "interações medicamentosas" in texto:
        chunk.metadata["categoria"] = "interacoes"

    # Posologia e modo de uso
    elif "dose" in texto or "posologia" in texto or "como devo usar" in texto:
        chunk.metadata["categoria"] = "posologia_modo_uso"

    # Reações adversas
    elif "reações adversas" in texto or "quais os males" in texto:
        chunk.metadata["categoria"] = "reacoes_adversas"

    # Armazenamento
    elif "onde, como e por quanto tempo posso guardar" in texto or "armazenar" in texto:
        chunk.metadata["categoria"] = "armazenamento"

    # Superdosagem
    elif "quantidade maior do que a indicada" in texto or "superdosagem" in texto:
        chunk.metadata["categoria"] = "superdosagem"

    # Conteúdo geral / administrativo
    else:
        chunk.metadata["categoria"] = "geral"



In [9]:
import random

# Seleciona dois chunks aleatórios
chunks_aleatorios = random.sample(chunks, 2)

# Imprime os metadados e um trecho do conteúdo
for i, chunk in enumerate(chunks_aleatorios, start=1):
    print(f"\n--- Chunk Aleatório {i} ---")
    print("Metadados:")
    print(chunk.metadata)
    print("\nConteúdo (início):")
    print(chunk.page_content[:300])



--- Chunk Aleatório 1 ---
Metadados:
{'source': 'dipirona.pdf', 'page': 3, 'medicamento': 'dipirona', 'categoria': 'como_funciona'}

Conteúdo (início):
operar carros ou máquinas), especialmente quando álcool foi consumido. 
 
Reações anafiláticas/anafilactóides (reação alérgica grave e imediata que pode levar à morte): 
Caso você esteja em alguma das situações abaixo, converse com seu médico, pois estas situações 
apresentam risco especial para pos

--- Chunk Aleatório 2 ---
Metadados:
{'source': 'dipirona.pdf', 'page': 6, 'medicamento': 'dipirona', 'categoria': 'como_funciona'}

Conteúdo (início):
Distúrbios renais e urinários 
Em casos muito raros, especialmente em pacientes com histórico de doença nos ri ns, pode ocorrer piora 
súbita ou recente da função dos rins (insuficiência renal aguda), em alguns casos com diminuição da 
produção de urina, redução muito acentuada da produção de urina 



## 5️⃣ Geração de Embeddings e Banco Vetorial


In [10]:
!pip install chromadb


In [11]:
# Embeddings (forma correta e atual)
from langchain_openai import OpenAIEmbeddings

# Vector Store
from langchain_community.vectorstores import Chroma

# Inicializa o modelo de embeddings (forma atual)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # opcional, mas recomendado
)

# Cria o banco vetorial com os chunks
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_bulas"
)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************SdsA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}


## 6️⃣ Recuperação de Contexto (Retriever)


In [ ]:

# Cria o retriever para busca semântica
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # Número de chunks retornados
)



## 7️⃣ Integração com Pipeline RAG


In [ ]:

# Inicializa o modelo de linguagem
llm = ChatOpenAI(
    model="gpt-4o-mini",
)

# Cria a cadeia RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)



## 8️⃣ Testes e Validação das Respostas


In [ ]:
# Pergunta de teste
pergunta = "Quais são as contraindicações da dipirona?"

# Executa a pergunta no agente RAG (forma atual)
resposta = qa_chain.invoke(pergunta)

print("Pergunta:")
print(pergunta)

print("\nResposta do Agente:")
print(resposta["result"])

print("\nTrechos utilizados como contexto:\n")

# Percorre os documentos recuperados
for i, doc in enumerate(resposta["source_documents"], start=1):
    print(f"--- Trecho {i} ---")

    # Metadados principais
    print(f"Medicamento: {doc.metadata.get('medicamento', 'N/A')}")
    print(f"Categoria: {doc.metadata.get('categoria', 'N/A')}")
    print(f"Documento: {doc.metadata.get('source', 'Documento desconhecido')}")
    print(f"Página: {doc.metadata.get('page', 'N/A')}")

    # Conteúdo recuperado
    print("\nConteúdo do chunk:")
    print(doc.page_content)
    print("\n")


In [ ]:
# Pergunta de teste 1
pergunta = "Qual é a posologia recomendada do paracetamol para adultos?"

# Executa a pergunta no agente RAG
resposta = qa_chain.invoke(pergunta)

print("Pergunta:")
print(pergunta)

print("\nResposta do Agente:")
print(resposta["result"])

print("\nTrechos utilizados como contexto:\n")

for i, doc in enumerate(resposta["source_documents"], start=1):
    print(f"--- Trecho {i} ---")
    print(f"Medicamento: {doc.metadata.get('medicamento', 'N/A')}")
    print(f"Categoria: {doc.metadata.get('categoria', 'N/A')}")
    print(f"Documento: {doc.metadata.get('source', 'Documento desconhecido')}")
    print(f"Página: {doc.metadata.get('page', 'N/A')}")
    print("\nConteúdo do chunk:")
    print(doc.page_content)
    print("\n")



## ✅ Conclusão

Neste Projeto 2, evoluímos para um RAG mais próximo de cenários reais,
com maior controle sobre:
- Qualidade semântica
- Organização dos documentos
- Explicabilidade das respostas

Esse padrão pode ser aplicado em sistemas de:
- Saúde
- Jurídico
- Compliance
- RH
